In [51]:
using CSV
import Pkg
Pkg.add("JSON3")
using JSON3
using DataFrames


# Read the Locations CSV file
df = CSV.read("Locations_Capabilities.csv", DataFrame)

# Function to parse JSON strings into Julia objects
function parse_json_string(str)
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    return JSON3.read(str)
end

# Process each location's data
locations_data = Dict()

for row in eachrow(df)
    location_id = row.Location_ID
    
    # Process workers data
    workers = parse_json_string(row.Workers)
    
    # Process machines data
    machines = parse_json_string(row.Machines)
    
    # Process inventory resources
    inventory = parse_json_string(row.Inventory_Resources)
    
    # Store all data for this location
    locations_data[location_id] = Dict(
        "workers" => workers,
        "machines" => machines,
        "inventory" => inventory
    )
end

# Create helper functions
function get_all_location_ids()
    return locations_data.keys()
end

function get_all_workers_ids(location_id)
    return locations_data[location_id]["workers"].keys()
end

function get_worker_capabilities(location_id, worker_id)
    return locations_data[location_id]["workers"][worker_id]["Capabilities"]
end

function get_all_machines_ids(location_id)
    return locations_data[location_id]["machines"].keys()
end

function get_machine_capabilities(location_id, machine_id)
    return locations_data[location_id]["machines"][machine_id]["Capabilities"]
end

function get_all_resources_ids(location_id)
    return locations_data[location_id]["inventory"].keys()
end

function get_resource_amounts(location_id, resource_id)
    return locations_data[location_id]["inventory"][resource_id]
end

println(locations_data["L1"]["machines"][1])


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


{
       "Machine_ID": "L1_M1",
     "Capabilities": "Painting",
   "Breakdown_Prob": 0.11
}


In [46]:
using CSV
using JSON3
using DataFrames

# Read the CSV file
df = CSV.read("Projects_Data_with_Stage_Durations.csv", DataFrame)

# Function to clean and parse JSON strings into Julia objects
function parse_json_string(str)
    # Clean the string: remove newlines and extra spaces
    str = replace(str, r"\s+" => " ")  # Replace multiple spaces with single space
    str = replace(str, "\n" => "")     # Remove newlines
    str = strip(str)                   # Remove leading/trailing whitespace
    
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    
    try
        return JSON3.read(str)
    catch e
        println("Error parsing JSON string: ", str)
        rethrow(e)
    end
end

# Process each project's data
projects_data = Dict()

for row in eachrow(df)
    project_id = strip(row.Project_ID)
    
    # Process all JSON fields
    total_resources = parse_json_string(row.Total_Resources)
    workers_req = parse_json_string(row.Workers_Requirement)
    machines_req = parse_json_string(row.Machines_Requirement)
    stages = parse_json_string(row.Stages)
    
    # Store all data for this project
    projects_data[project_id] = Dict(
        "total_resources" => total_resources,
        "workers_requirement" => workers_req,
        "machines_requirement" => machines_req,
        "deadline" => row.Deadline,
        "customer_priority" => row.Customer_Priority,
        "stages" => stages,
        "revenue" => row.Revenue
    )
end

# Create helper functions
function get_project_data(project_id)
    return projects_data[project_id]
end

function get_all_project_ids()
    return collect(keys(projects_data))
end

function get_project_stages(project_id)
    return projects_data[project_id]["stages"]
end

function get_project_resources(project_id)
    return projects_data[project_id]["total_resources"]
end

function get_project_stage_workers(project_id, stage_id)
    stages = projects_data[project_id]["stages"]
    return stages[stage_id]["Workers_Needed"]
end

function get_project_stage_machines(project_id, stage_id)
    stages = projects_data[project_id]["stages"]
    return stages[stage_id]["Machines_Needed"]
end

function get_project_stage_duration(project_id, stage_id)
    stages = projects_data[project_id]["stages"]
    return stages[stage_id]["Duration_days"]
end

function get_project_deadline(project_id)
    return projects_data[project_id]["deadline"]
end

function get_project_priority(project_id)
    return projects_data[project_id]["customer_priority"]
end

function get_project_revenue(project_id)
    return projects_data[project_id]["revenue"]
end
 

# Print some data to verify it worked
first_project_id = get_all_project_ids()[1]
second_project_id = get_all_project_ids()[2]
println("First project data:")
println("Project ID: ", first_project_id)
println("Deadline: ", get_project_deadline(first_project_id))
println("Priority: ", get_project_priority(first_project_id))
println("Revenue: ", get_project_revenue(first_project_id))
println("Total Resources: ", get_project_resources(first_project_id))
println("First Stage: ", get_project_stages(first_project_id)[1])
println("Second Stage: ", get_project_stages(second_project_id)[1])

First project data:
Project ID: P13
Deadline: 66
Priority: 1
Revenue: 16000
Total Resources: {
   "Plastic": 66,
     "Steel": 51,
     "Paint": 35
}
First Stage: {
           "Stage_ID": 1,
    "Machines_Needed": [
                         "Inspector"
                       ],
     "Workers_Needed": [
                         "Painter",
                         "Inspector"
                       ],
   "Resources_Needed": {
                          "Plastic": 47,
                            "Steel": 27,
                            "Paint": 13
                       },
      "Duration_days": 8
}
Second Stage: {
           "Stage_ID": 1,
    "Machines_Needed": [
                         "Assembler",
                         "Packer"
                       ],
     "Workers_Needed": [
                         "Painter"
                       ],
   "Resources_Needed": {
                          "Plastic": 14,
                            "Steel": 23,
                            "Paint": 20

In [60]:
#Preparing the rest of the Parameters

import Pkg; Pkg.add("GLPK")
using JuMP, GLPK, DataFrames, CSV

########################################################
# Important Parameter, this is the end of the job scheduler
deadline = 100
########################################################

# Get all locations
locations = collect(keys(locations_data))

# Create location-specific sets for workers and machines
location_workers = Dict(
    loc => collect(1:length(locations_data[loc]["workers"]))
    for loc in locations
)

location_machines = Dict(
    loc => collect(1:length(locations_data[loc]["machines"]))
    for loc in locations
)

# Get all projects and their stages
projects = get_all_project_ids()
project_stages = Dict(
    p => collect(1:length(get_project_stages(p))) 
    for p in projects
)

T = 1:deadline




   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


1:100

In [64]:
#Preparing the rest of the Parameters
using JuMP, GLPK

# Create model
model = Model(GLPK.Optimizer)

# Decision variables - now with location-specific workers and machines
@variable(model, y[p in projects,
               s in project_stages[p],
               l in locations,
               m in location_machines[l],
               t in T], Bin)  # Machine assignments

@variable(model, z[p in projects,
               s in project_stages[p],
               l in locations,
               w in location_workers[l],
               t in T], Bin)  # Worker assignments

@variable(model, x[projects], Bin)  # Project acceptance
@variable(model, w[projects, T], Bin)  # Project start times

@variable(model, stage_start[p in projects, s in project_stages[p]], Int)

# Objective function
@objective(model, Max, sum(get_project_revenue(p) * x[p] for p in projects))

# Constraints
# Machine assignment constraints - now per location
for l in locations
    for m in location_machines[l], t in T
        @constraint(model, sum(y[p,s,l,m,t] 
                         for p in projects, 
                         s in project_stages[p]) <= 1)
    end
end

# Worker assignment constraints - now per location
for l in locations
    for w in location_workers[l], t in T
        @constraint(model, sum(z[p,s,l,w,t] 
                         for p in projects, 
                         s in project_stages[p]) <= 1)
    end
end

for l in locations, p in projects, s in project_stages[p], t in T
    # Machine capability constraints
    stage_machines_needed = get_project_stage_machines(p, s)
    for m in location_machines[l]
        machine_capabilities = get_machine_capabilities(l,m)
        
        if isempty(intersect(stage_machines_needed, machine_capabilities)) ####Ensures if the machine has a list of skills, there still exists no identical skill in the stage
            @constraint(model, y[p,s,l,m,t] == 0)
        end
    end

    # Worker capability constraints
    stage_workers_needed = get_project_stage_workers(p, s)
    for w in location_workers[l]
        worker_capabilities = get_worker_capabilities(l,w)
        if isempty(intersect(worker_capabilities, stage_workers_needed))
            @constraint(model, z[p,s,l,w,t] == 0)
        end
    end
end

# Project acceptance and timing constraints
for p in projects
    # Project must start once if accepted
    @constraint(model, sum(w[p,t] for t in T) == x[p])
    
    # Link project start time to first stage
    @constraint(model, stage_start[p,1] == sum(t * w[p,t] for t in T))
    
    # Stage sequence constraints with durations
    for s in project_stages[p]
        if s > 1
            # Each stage must start after the previous stage ends
            duration_prev_stage = get_project_stage_duration(p, s-1)
            @constraint(model, stage_start[p,s] >= stage_start[p,s-1] + duration_prev_stage)
        end
        
        # Stage must complete before deadline if project is accepted
        duration_current_stage = get_project_stage_duration(p, s)
        @constraint(model, stage_start[p,s] + duration_current_stage <= get_project_deadline(p))
    end
end

# Link stage start times to worker and machine assignments
for p in projects, s in project_stages[p], l in locations
    # Workers must be assigned at stage start time
    for w in location_workers[l]
        @constraint(model, sum(t * z[p,s,l,w,t] for t in T) == stage_start[p,s])
    end
    
    # Machines must be assigned at stage start time
    for m in location_machines[l]
        @constraint(model, sum(t * y[p,s,l,m,t] for t in T) == stage_start[p,s])
    end
end

optimize!(model)
for p in projects
    println("Project: ", p, " ", value(x[p]))
end


MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.